In [1]:
import pandas as pd

In [2]:
df1 = pd.read_csv('/opt/ml/detection/ensemble_dir/lee_6669.csv')
df2 = pd.read_csv('/opt/ml/detection/ensemble_dir/yolov5_submission_detect_exp5.csv')
print(len(df1), len(df2))

4871 4871


## 경향성 분석
- conf score와 box 크기와의 상관관계 -> pseudo labeling에서 conf 크게해서 가져오면 큰 box만 가져오지 않을까?를 알아봐야 됨.
- 성능 좋은 csv들의 특징을 알아봐야 됨. -> 성능 구린 친구들과 비교해보자
- 각 이미지별 예측한 카테고리가 얼마나 동일한가? ex) test/000.jpg에 대해서 df1은 'Paper', 'General Trash' / df2는 'Paper' 만 예측 -> df1을 기준으로 df2와 50% 동일

In [3]:
# 이미지 번호, conf score, box 크기 가져오기
bbox = df1['PredictionString'].values.tolist()
# len(bbox)

conf_score = []
box_area = []
img_num = []
for img_idx,box in enumerate(bbox):

    lst = []
    if type(box) == float:
        continue
    for i,b in enumerate(box.split(" ")):
        if i%6 == 0 and i != 0:
            img_num.append(img_idx)
            conf_score.append(float(lst[1]))
            box_area.append((float(lst[4]) - float(lst[2])) * (float(lst[5]) - float(lst[3])))
            lst = []
        lst.append(b)

print(len(img_num), len(conf_score),  len(box_area))

# 이미지 번호, conf score, box 크기 가져오기
bbox2 = df2['PredictionString'].values.tolist()
# len(bbox)

conf_score_2 = []
box_area_2 = []
img_num_2 = []
for img_idx,box in enumerate(bbox2):

    lst = []
    if type(box) == float:
        continue
    for i,b in enumerate(box.split(" ")):
        if i%6 == 0 and i != 0:
            img_num_2.append(img_idx)
            conf_score_2.append(float(lst[1]))
            box_area_2.append((float(lst[4]) - float(lst[2])) * (float(lst[5]) - float(lst[3])))
            lst = []
        lst.append(b)

print(len(img_num_2), len(conf_score_2),  len(box_area_2))

249891 249891 249891
45157 45157 45157


In [7]:

df = pd.DataFrame()
df['conf_1'] = conf_score
df['box_area_1'] = box_area
df['img_num_1'] = img_num

df_compare = pd.DataFrame()
df_compare['conf_2'] = conf_score_2
df_compare['box_area_2'] = box_area_2
df_compare['img_num_2'] = img_num_2

df.head()
print(df)

          conf_1     box_area_1  img_num_1
0       0.057813     577.450359          0
1       0.047805    3105.631363          0
2       0.992475   48392.862863          0
3       0.304816     523.329195          0
4       0.156587   23251.866890          0
...          ...            ...        ...
249886  0.001629  159669.665492       4870
249887  0.999470  163762.725787       4870
249888  0.005326  165475.685269       4870
249889  0.003371    1433.645418       4870
249890  0.001660    1314.220194       4870

[249891 rows x 3 columns]


In [6]:
df[df['conf_1'] > 0.5]

,conf_1,box_area_1,img_num_1
2,0.992475,48392.862863,0
19,0.967123,3837.468225,0
20,0.672119,5712.260205,0
21,0.668740,8245.977056,0
22,0.637285,16235.312069,0
...,...,...,...
249747,0.999507,141753.464367,4867
249774,0.995964,235110.227593,4868
249775,0.812237,86387.101353,4868
249783,0.992006,752699.090476,4869


In [85]:
# 작은 box?
df[df['box_area_1'] < 40000]

# r, p_value = stats.pearsonr(df[df['box_area_1'] < 40000]['conf_1'], df[df['box_area_1'] < 40000]['box_area_1'])
# print(f"Correlation coefficient = {r}")

,conf_1,box_area_1,img_num_1
0,0.057813,577.450359,0
1,0.047805,3105.631363,0
3,0.304816,523.329195,0
4,0.156587,23251.866890,0
5,0.112133,8827.051359,0
...,...,...,...
249883,0.004046,1363.611722,4870
249884,0.001429,1139.470798,4870
249885,0.001267,352.145492,4870
249889,0.003371,1433.645418,4870


In [65]:
import scipy.stats as stats
import numpy as np

# The stats.pearsonr() returns 2 values the correlation coefficient and the p-value
r, p_value = stats.pearsonr(df['conf_1'], df['box_area_1'])
print(f"Correlation coefficient = {r}")

Correlation coefficient = 0.17324027570717054
